In [1]:
import dash
# import dash_core_components as dcc
from dash import dcc
# import dash_html_components as html
from dash import html
import dash_bootstrap_components as dbc

In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import date

In [3]:
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.MATERIA])

INCIDENT TICKETS

In [4]:
# Create initial Dataframe
df_incident = pd.read_csv('/Users/noel.c.joaquin/Desktop/BASE_DATA/INC_YTD_nov2022.csv')
df_incident.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   number                 83 non-null     object
 1   state                  83 non-null     object
 2   opened_at              83 non-null     object
 3   resolved_at            75 non-null     object
 4   closed_at              74 non-null     object
 5   u_research_assessment  79 non-null     object
dtypes: object(6)
memory usage: 4.0+ KB


In [5]:
# Replace NaN values with '01-01-2200' 
df_incident['resolved_at'].fillna('1/1/2200', inplace=True)
df_incident['closed_at'].fillna('1/1/2200', inplace=True)
df_incident.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   number                 83 non-null     object
 1   state                  83 non-null     object
 2   opened_at              83 non-null     object
 3   resolved_at            83 non-null     object
 4   closed_at              83 non-null     object
 5   u_research_assessment  79 non-null     object
dtypes: object(6)
memory usage: 4.0+ KB


In [6]:
# Save new Dataframe 
df_incident.to_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', index=False)

In [7]:
# Recreate Dataframe and conver Date columns to Datetime format
d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/INC_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at', 'resolved_at'],
                            date_parser = d_parser , encoding = 'unicode_escape')
                            
df_task.head()

C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20592\1630154363.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20592\1630154363.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')
C:\Users\noel.c.joaquin\AppData\Local\Temp\ipykernel_20592\1630154363.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')


,number,state,opened_at,resolved_at,closed_at,u_research_assessment
0,INC1234577,Resolved,2022-01-01,2022-01-02,2022-02-01,Data Issue
1,INC1234578,Resolved,2022-01-02,2022-01-03,2022-02-02,Login Issue
2,INC1234579,Resolved,2022-01-03,2022-01-04,2022-02-03,Product Issue
3,INC1234580,Resolved,2022-01-04,2022-01-05,2022-02-04,System Issue
4,INC1234581,Resolved,2022-01-05,2022-01-06,2022-02-05,Workspace Issue


In [21]:
month_date_today = date.today()
split_month_date_today = str(month_date_today).split('-')
month_first_day = str(month_date_today).replace(str(f"-{split_month_date_today[-1]}"), str(f"-01")) 

CARD COUNTS - INC Opened

In [22]:
def inc_opened_count(timeline):    
    count = 0

    if timeline == 'current-month':
        date_filter_inc_currmonth = (df_task['opened_at'] >= month_first_day)
        count = len(date_filter_inc_currmonth.loc[date_filter_inc_currmonth])
        return count
    elif timeline == 'last-month':
        return count
    elif timeline == 'current-year':
        year_first_day = month_first_day.replace(str(f"-{split_month_date_today[1]}"), str(f"-01"))
        count = df_task.loc[df_task['opened_at'] >= year_first_day].shape[0]
        return count
    elif timeline == 'ytd':
        count = df_task.shape[0]
        return count
        

In [23]:
inc_ocm = inc_opened_count('current-month')
inc_olm = inc_opened_count('last-month')
inc_ocy = inc_opened_count('current-year')
inc_oytd = inc_opened_count('ytd')

print(f"inc_ocm: {inc_ocm}")
print(f"inc_olm: {inc_olm}")
print(f"inc_ocy: {inc_ocy}")
print(f"inc_oytd: {inc_oytd}")

inc_ocm: 9
inc_olm: 0
inc_ocy: 43
inc_oytd: 83


CARD COUNTS - INC Resolved

In [24]:
def inc_resolved_count(timeline):
    count = 0

    if timeline == 'current-month':
        date_filter_inc_currmonth = (df_task['resolved_at'] >= month_first_day) & (df_task['resolved_at'] != '2200-01-01')
        count = len(date_filter_inc_currmonth.loc[date_filter_inc_currmonth])
        return count
    elif timeline == 'last-month':
        return count
    elif timeline == 'current-year':
        year_first_month = month_first_day.replace(str(f"-{split_month_date_today[1]}"), str(f"-01"))
        date_filter_inc_curryear = (df_task['resolved_at'] >= year_first_month) & (df_task['resolved_at'] != '2200-01-01')
        count = len(date_filter_inc_curryear.loc[date_filter_inc_curryear])
        return count
    elif timeline == 'ytd':
        total_closed_count = df_task.loc[df_task['state'] == 'Closed'].shape[0]
        total_resolved_count = df_task.loc[df_task['state'] == 'Resolved'].shape[0]
        count = total_closed_count + total_resolved_count
        return count

In [25]:
inc_rcm = inc_resolved_count('current-month')
inc_rlm = inc_resolved_count('last-month')
inc_rcy = inc_resolved_count('current-year')
inc_rytd = inc_resolved_count('ytd')

print(f"inc_rcm: {inc_rcm}")
print(f"inc_rlm: {inc_rlm}")
print(f"inc_ocy: {inc_rcy}")
print(f"inc_rytd: {inc_rytd}")

inc_rcm: 1
inc_rlm: 0
inc_ocy: 37
inc_rytd: 75


Volume of Incidents by Issue Type

In [ ]:
# Get all 'Research Assessment' types
research_assessment_types = df_task["u_research_assessment"].unique().tolist()

print(research_assessment_types)

In [ ]:
# Get Count for each 'Research Assessment'

research_assessment_count = []

for issue_type in research_assessment_types:
    count = df_task.loc[df_task["u_research_assessment"] == issue_type].shape[0]
    research_assessment_count.append(count)

    print(f"issue_type: {issue_type} - {count}")

print(research_assessment_count)

In [ ]:
# Create DataFrame for Research Assessment Count
df_research_assessment = pd.DataFrame({
    "Research Assessment": research_assessment_types,
    "": research_assessment_count
})

df_research_assessment = df_research_assessment.sort_values('', ascending=False)

df_research_assessment

In [ ]:
# Make Bar Chart for df_research_assessment
fig = px.bar(df_research_assessment, x="Research Assessment", y="", barmode="group", text_auto=True)
fig.show()

CARD

In [ ]:
first_card = dbc.Card(
    dbc.CardBody(
        [
            html.H5("Incident Tickets (Opened)- Current Month", className="card-title", style={'font-size': '1.5rem'}),
            html.P(currmonth_opened_ticket, style={'font-size': '5rem', 'font-weight': 'bold'}),
        ]
    )
)

second_card = dbc.Card(
    dbc.CardBody(
        [
            html.H5("Incident Tickets(Resolved) - Current Month", className="card-title", style={'font-size': '1.5rem'}),
            html.P(currmonth_resolved_ticket, style={'font-size': '5rem', 'font-weight': 'bold'}),
        ]
    )
)

third_card = dbc.Card(
    dbc.CardBody(
        [
            html.H5("Incident Volume - Closed", className="card-title", style={'font-size': '1.5rem'}),
            html.P(total_closed_count, style={'font-size': '5rem', 'font-weight': 'bold'}),
        ]
    )
)

fourth_card = dbc.Card(
    dbc.CardBody(
        [
            html.H5("Incident Volume - Resolved - Current Month", className="card-title", style={'font-size': '1.5rem'}),
            html.P(total_resolved_count, style={'font-size': '5rem', 'font-weight': 'bold'}),
        ]
    )
),

cards = dbc.Row(
    [
        dbc.Col(first_card, width = 3),
        dbc.Col(second_card, width = 3),
        dbc.Col(third_card, width = 3),
        dbc.Col(fourth_card, width = 3),
    ]
)

AP Incident Tickets Opened and Resolved

In [ ]:
# Get distinct values of Year-Month from all data and store in a list

selected_year = '2022'

initial_year_month_list = df_task['opened_at'].dt.strftime('%Y-%m').unique().tolist()
year_month_list = []

for x in initial_year_month_list:
    if selected_year in x:
        year_month_list.append(x)

print(year_month_list)

# Get available month names for selected year
get_month_names = df_task.loc[df_task['opened_at'] > selected_year]['opened_at'].dt.strftime('%b').unique().tolist()
print(f"get_month_names: {get_month_names}")

In [ ]:
# Get Opened Ticket Count for each Year-Month
year_month_count_opened = []

for get_curr_year_month in year_month_list:

    get_curr_month = get_curr_year_month.split('-')[1]
    get_curr_year = get_curr_year_month.split('-')[0]
    get_next_month = int(get_curr_month) + 1
    get_next_year = int(get_curr_year) + 1

    if get_next_month > 12:
        
        get_next_month = 1;
        get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))
        get_next_year_month = get_next_year_month.replace(str(get_curr_year), str(get_next_year))

        count = df_task.loc[(df_task['opened_at'] < f"{get_next_year_month}-01") & 
                                (df_task['opened_at'] >= f"{get_curr_year_month}-01")]

        year_month_count_opened.append(count.shape[0])
    else:
        get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))

        count = df_task.loc[(df_task['opened_at'] < f"{get_next_year_month}-01") & 
                                (df_task['opened_at'] >= f"{get_curr_year_month}-01")]

        year_month_count_opened.append(count.shape[0])

print(year_month_count_opened)

In [ ]:
# Get Resolved Ticket Count for each Year-Month
year_month_count_resolved = []

for get_curr_year_month in year_month_list:

    get_curr_month = get_curr_year_month.split('-')[1]
    get_curr_year = get_curr_year_month.split('-')[0]
    get_next_month = int(get_curr_month) + 1
    get_next_year = int(get_curr_year) + 1

    if get_next_month > 12:
        
        get_next_month = 1;
        get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))
        get_next_year_month = get_next_year_month.replace(str(get_curr_year), str(get_next_year))

        count = df_task.loc[(df_task['resolved_at'] < f"{get_next_year_month}-01") & 
                                (df_task['resolved_at'] >= f"{get_curr_year_month}-01")]

        year_month_count_resolved.append(count.shape[0])
    else:
        get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))

        count = df_task.loc[(df_task['resolved_at'] < f"{get_next_year_month}-01") & 
                                (df_task['resolved_at'] >= f"{get_curr_year_month}-01")]

        year_month_count_resolved.append(count.shape[0])

print(year_month_count_resolved)

In [ ]:
# Create DataFrame for 'opened_at' column based on 'year_month_list' and 'year_month_count'
df_inc_open_and_close = pd.DataFrame({
    "month_year": year_month_list,
    "count_open": year_month_count_opened,
    "count_resolved": year_month_count_resolved
})

df_inc_open_and_close

In [ ]:
# Create figure for 'df_inc_open_and_close'
colors = px.colors.qualitative.Set3

fig2 = go.Figure(data=[
    go.Bar(name='Opened', x=get_month_names, y=year_month_count_opened, text=year_month_count_opened, marker=dict(color=colors[4])),
    go.Bar(name='Resolved', x=get_month_names, y=year_month_count_resolved, text=year_month_count_resolved, marker=dict(color=colors[3])),
])

fig2.show()

TASK TICKETS

In [ ]:
# Fill any null values
df_task['closed_at'].fillna('1/1/2200', inplace=True)
df_task.info()

In [ ]:
# Create SCTASK DataFrame

d_parser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y')

df_task = pd.read_csv('/Users/noel.c.joaquin/Desktop/Python/TASK_YTD_nov2022.csv', parse_dates=['opened_at', 'closed_at'], date_parser = d_parser, encoding='unicode_escape')

df_task.head()

In [ ]:
# Get Opened Task Count for each Year-opened_task = []

year_month_count_opened_task = []

for get_curr_year_month in year_month_list:

    get_curr_month = get_curr_year_month.split('-')[1]
    get_curr_year = get_curr_year_month.split('-')[0]
    get_next_month = int(get_curr_month) + 1
    get_next_year = int(get_curr_year) + 1

    if get_next_month > 12:
        
        get_next_month = 1;
        get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))
        get_next_year_month = get_next_year_month.replace(str(get_curr_year), str(get_next_year))

        count = df_task.loc[(df_task['opened_at'] < f"{get_next_year_month}-01") & 
                                (df_task['opened_at'] >= f"{get_curr_year_month}-01")]

        year_month_count_opened_task.append(count.shape[0])
    else:
        get_next_year_month = get_curr_year_month.replace(str(f"-{get_curr_month}"), str(f"-{get_next_month}"))

        count = df_task.loc[(df_task['opened_at'] < f"{get_next_year_month}-01") & 
                                (df_task['opened_at'] >= f"{get_curr_year_month}-01")]

        year_month_count_opened_task.append(count.shape[0])

print(year_month_count_opened_task)

In [ ]:
# Create DataFrame
df_inc_and_task_open = pd.DataFrame({
    "month_year": year_month_list,
    "count_open_inc": year_month_count_opened,
    "count_open_task": year_month_count_opened_task
})

print(df_inc_and_task_open)

In [ ]:
# Create figure for 'df_inc_and_task_open'
colors = px.colors.qualitative.Set3

fig3 = go.Figure(data=[
    go.Bar(name='Incidents', x=get_month_names, y=year_month_count_opened, text=year_month_count_opened, marker=dict(color=colors[4])),
    go.Bar(name='Tasks', x=get_month_names, y=year_month_count_opened_task, text=year_month_count_opened_task, marker=dict(color=colors[3])),
])

fig3

In [ ]:
# Filler Pie Chart
df_pie = px.data.tips()
fig_pie = px.pie(df_pie, values='tip', names='day', color='day',
             color_discrete_map={'Thur':'lightcyan',
                                 'Fri':'cyan',
                                 'Sat':'royalblue',
                                 'Sun':'darkblue'})
fig_pie.show()

In [ ]:
# Create server variable with Flask server object for use with gunicorn
server = app.server

# Construct App Layout
app.layout = html.Div(children=[
    html.H1('Analytics Platform Ticket Dashboard'),
    dcc.Tabs(id="tabs-content-input", value='current-month', children=[
        dcc.Tab(label='Current Month', value='current-month'),
        dcc.Tab(label='Last Month', value='last-month'),
        dcc.Tab(label='Current Year', value='current-year'),
        dcc.Tab(label='Year to Date', value='ytd')
    ]),
    html.Div(id='tabs-content-output')
])

@app.callback(Output('tabs-content-output', 'children'),
              Input('tabs-content-input', 'value'))
def render_content(tab):
    if tab == 'current-month':
        return html.Div([

        ])
    elif tab == 'last-month':
        return html.Div([

        ])
    elif tab == 'current-year':
        return html.Div([

        ])
    elif tab == 'ytd':
        return html.Div([

        ])

app.run_server()